Import các thư viện cần sử dụng.

sklearn để sử dụng thuật toán SVM

cv2 để xử lí ảnh

os, np để đọc file và tính toán trên các mảng dữ liệu

In [1]:
from sklearn.svm import LinearSVC
from skimage import exposure
from skimage import feature
import argparse
import cv2
import imutils
from imutils import paths
import os
import numpy as np

In [2]:
print("Extracting features...")
data = []
labels = []

Extracting features...


In [3]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [4]:
img_person = load_images_from_folder("is_Face")
print("Number of photos are face : ",len(img_person))
sample = img_person[2]
cv2.imshow("No_name", sample)
cv2.waitKey(0)

Number of photos are face :  178


-1

In [5]:
img_not_person = load_images_from_folder("not_Face")
print("Number of photos are not face : ",len(img_not_person))
sample = img_not_person[2]
cv2.imshow("No_name", sample)
cv2.waitKey(0)

Number of photos are not face :  130


-1

In [6]:
for image in img_not_person:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_resize = cv2.resize(gray, (150,150))
    H, hogImage = feature.hog(img_resize, orientations=9, pixels_per_cell=(10,10), cells_per_block=(2,2), transform_sqrt=True, block_norm="L1", visualize=True)
    data.append(H)
    labels.append(0)
for image in img_person:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_resize = cv2.resize(gray, (150,150))
    H, hogImage = feature.hog(img_resize, orientations=9, pixels_per_cell=(10,10), cells_per_block=(2,2), transform_sqrt=True, block_norm="L1", visualize=True)
    data.append(H)
    labels.append(1)

In [7]:
data = np.asanyarray(data)
labels = np.asanyarray(labels)
print("Size of data to train : ",data.shape)
print("Size of labels to train : ", labels.shape)

Size of data to train :  (308, 7056)
Size of labels to train :  (308,)


Bây giờ chúng ta tạo model với thuật toán SVM cho bộ dữ liệu trên.



In [8]:
print("Training classifier...")
model = LinearSVC()
model.fit(data, labels)
print("Evaluting...")

Training classifier...
Evaluting...


In [9]:
from sklearn import svm
clf = svm.SVC().fit(data, labels)
y_pred = clf.predict(data)
from sklearn.metrics import accuracy_score
print ("Xac xuat sử dụng thư viện : %.2f %%" %(100*accuracy_score(labels, y_pred.tolist())) )

C:\Users\trung\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Xac xuat sử dụng thư viện : 57.79 %


In [10]:
y_pred = model.predict(data)
from sklearn.metrics import accuracy_score
print ("Xac xuat sử dụng thư viện : %.2f %%" %(100*accuracy_score(labels, y_pred.tolist())) )

Xac xuat sử dụng thư viện : 100.00 %


Chúng tau thử qua với 2 model khác nhau của SVM tau sử dụng model đầu với độ chính xác cao hơn.


Bước tiếp theo bây giờ khi import một ảnh mới vào chúng ta cần làm : 
+ Sử dụng tập các của số trượt có kích thước khác nhau để trượt lên bức ảnh từ trái sang phải từ trên xuống dưới.
+ Với mỗi cửa số trượt như vậy ta thu được một hình ảnh chúng tau resize kích thước ảnh đó theo đúng kích thước của dữ liệu train
+ Lấy đặc trưng HOG của ảnh và sau đó cho model xử lí xem nó có đúng hay không.
+ Nếu đúng thì chúng ta vẽ bao quanh khu vực đó lại.


In [11]:
test = cv2.imread("hai.jpg")
gray = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)

In [12]:
test = cv2.resize(test, (150,150))
H, hogImage = feature.hog(test, orientations=9, pixels_per_cell=(10,10), cells_per_block=(2,2), 
                          transform_sqrt=True, block_norm="L1", visualize=True)
predict = model.predict(H.reshape(1,-1))
print(predict)


[1]


Quan sát bức ảnh test tôi thấy ảnh hơi chênh sáng giữa các vừng sáng tối nên tôi quyết định cân bằng lại độ tương phản và ánh sáng của bức ảnh. Rất may việc đó đã được thực hiện trong thư viện Open CV.

In [54]:
img_test = cv2.imread("Test\sample_test.jpeg")
image_resize_test = cv2.resize(img_test, (800,500))
orig_img = image_resize_test.copy()
cv2.imshow("equals", image_resize_test)
cv2.waitKey(0)

-1

Chúng ta sẽ làm việc với ảnh có size = (500, 700) pixel tìm và vẽ hình vuông cho vị trí có phát hiện khuôn mặt.

Tạo một số cửa số trượt có kích thước khác nhau để rò tìm từng vị trí

Trong Open CV hàm vẽ hình chữ nhật là cv2.rectangle()

các tham số trong đó bao gồm :
    + Vẽ trên một ảnh nào ( background)
    + Tọa độ của góc trên cùng bên trái
    + Tọa độ của góc dưới cùng bên phải 
    + Màu sắc (kênh RGB)
    + Kích thước của viền.

Nên tôi chỉ dùng độ dài đường chéo đặc trưng cho mỗi của sổ trượt.


In [55]:
print(image_resize_test.shape)
sliding_window = [70] # chúng ta nên để một vài giá trị này để phát hiện đối tượng với kích thước khác nhau
poss = []
stride = 10

(500, 800, 3)


Mỗi bức ảnh nếu chiếu theo hệ trục Oxy:
    + Gốc tọa độ tại góc trên cùng bên trái
    + Chiều dương Ox từ trái sáng phải
    + Chiều dường 0y từ trên xuống dưới.

In [50]:
def draw_border(x0, y0, d, img):
    cv2.rectangle(img, (x0,y0), (x0+d,y0+d), (0,0,255), 2)
    

In [56]:
img_paint = image_resize_test
x = 0
y = 0
d = 0
for denta in sliding_window:
    y = 0
    while y < image_resize_test.shape[0] :
        x = 0
        while x < image_resize_test.shape[1] :
            
            if x + denta < image_resize_test.shape[1]:
                d = d+1
                cut_img = image_resize_test[y : y + denta, x: x + denta]
                save_img = image_resize_test[y : y + denta, x: x + denta]
                cut_img_resize = cv2.resize(cut_img, (150,150))
                gray = cv2.cvtColor(cut_img_resize, cv2.COLOR_BGR2GRAY)
                H, hogImage = feature.hog(gray, orientations=9, pixels_per_cell=(10,10), cells_per_block=(2,2), 
                          transform_sqrt=True, block_norm="L1", visualize=True)
                pred = model.predict(H.reshape(1,-1))[0]
                
                if(pred == 1):
                    name = 'Save'
                    name = name + "\s" + str(d) + ".jpg";
                    # cv2.imwrite(name, save_img)
                    # draw_border(x, y, denta, img_paint)
                    poss.append([x, y , x + denta,  y + denta])
                    
            x = x + stride
        y = y + stride
cv2.imshow("result",img_paint)
cv2.waitKey(0)
print("Number sliding windown : ", d)
print("Posstion exact : ",poss)

Number sliding windown :  3650
Posstion exact :  [[340, 30, 410, 100], [340, 40, 410, 110], [340, 90, 410, 160], [340, 100, 410, 170], [340, 110, 410, 180], [280, 180, 350, 250], [460, 180, 530, 250], [470, 180, 540, 250], [380, 190, 450, 260], [390, 190, 460, 260], [470, 190, 540, 260], [200, 200, 270, 270], [200, 210, 270, 280]]


Chúng ta thấy có nhiều khu vực nhận dạng bởi các cửa số trùng lên nhau chúng ta cần chọn ra một cửa sổ tối ưu nhất cho việc phát hiện đối tượng. Tiếp tục chúng ta dùng thuật toán Non-Maximum Suppression for Object Detection.

Tất cả vị trí các khu vực phát hiện đã được lưu trong mảng poss chúng.

In [57]:
def non_maxium_suppression(boxes, overlapThresh):
    
    if len(boxes) == 0:
        return []
    pick = []
    
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    print("x1 : ", x1)
    print("y1 : ", y1)
    print("x2 : ", x2)
    print("y2 : ", y2)
    
    area = (x2 - x1 +1) * (y2 - y1 + 1) # diện tích của các hộp ( dài * rộng)
    
    print("area : ", area)
    idxs = np.argsort(y2) 
    
    print("idxs : ", idxs)
    while len(idxs) > 0:
         
        #lặp qua các tất cả các hộp    
        last = len(idxs) - 1
        i = idxs[last]
        # lấy hộp dưới cùng do đã sắp xếp 
        pick.append(i)  # lấy i
        suppress = [last]
        print("pick : ", pick)
        print("suppress : ", suppress)
        # loop over all indexes in the indexes list
        for pos in range(0, last):
            # grab the current index
            j = idxs[pos]
 
            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])
 
            # compute the width and height of the bounding box
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)
 
            # compute the ratio of overlap between the computed
            # bounding box and the bounding box in the area list
            overlap = float(w * h) / area[j]
 
                # if there is sufficient overlap, suppress the
                # current bounding box
            if overlap > overlapThresh:
                suppress.append(pos)
 
            # delete all indexes from the index list that are in the
            # suppression list
        idxs = np.delete(idxs, suppress)
        print("idxs : ", idxs)
        # return only the bounding boxes that were picked
    return boxes[pick]

In [58]:
poss = np.asanyarray(poss)
for [startX, startY, endX, endY] in poss:
    cv2.rectangle(orig_img, (startX, startY), (endX, endY), (0, 0, 255), 2)
result = non_maxium_suppression(poss, 0.3)    
for [startX, startY, endX, endY] in result:
    cv2.rectangle(image_resize_test, (startX, startY), (endX, endY), (0, 255, 0), 2)    
cv2.imshow("Original", orig_img)
cv2.imshow("After NMS", image_resize_test)
cv2.waitKey(0)

x1 :  [340 340 340 340 340 280 460 470 380 390 470 200 200]
y1 :  [ 30  40  90 100 110 180 180 180 190 190 190 200 210]
x2 :  [410 410 410 410 410 350 530 540 450 460 540 270 270]
y2 :  [100 110 160 170 180 250 250 250 260 260 260 270 280]
area :  [5041 5041 5041 5041 5041 5041 5041 5041 5041 5041 5041 5041 5041]
idxs :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
pick :  [12]
suppress :  [12]
idxs :  [ 0  1  2  3  4  5  6  7  8  9 10]
pick :  [12, 10]
suppress :  [10]
idxs :  [0 1 2 3 4 5 8 9]
pick :  [12, 10, 9]
suppress :  [7]
idxs :  [0 1 2 3 4 5]
pick :  [12, 10, 9, 5]
suppress :  [5]
idxs :  [0 1 2 3 4]
pick :  [12, 10, 9, 5, 4]
suppress :  [4]
idxs :  [0 1]
pick :  [12, 10, 9, 5, 4, 1]
suppress :  [1]
idxs :  []


-1

Lưu kết quả vào mục Result


In [59]:
cv2.imwrite("Result\Original.jpg",orig_img)
cv2.imwrite("Result\After_NMS.jpg",image_resize_test)
cv2.destroyWindow()

True